In [117]:
import pandas as pd
from sklearn.svm import SVR
import numpy as np

def mape(y_pred, y_true):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [297]:
dataset = pd.read_csv("../data/preprocessed_input_interpolate_20min.csv")
#dataset['date'] = pd.to_datetime(dataset['date'])
del dataset['date']

# Only select columns which are useful
# For (1, 0, 'tot')
selected_col = ["hour", "precipitation", "dayofweek", "is_holiday", "hour", "hour", "hour", "hour", "(3, 1, 'privatecar')", 'hour', 'dayofweek', 'hour', 'is_holiday', 'hour', 'hour']
# For (1, 1, 'tot')
#selected_col = ["hour", "precipitation", "dayofweek", "is_holiday", "(1, 0, 'cargocar')", "(1, 0, 'cargocar')", "(1, 0, 'cargocar')", "hour", "(1, 0, 'cargocar')", "(1, 0, 'cargocar')", "(1, 0, 'cargocar')", "hour", "hour", "(1, 0, 'cargocar')", "hour", "(1, 0, 'cargocar')", "(1, 0, 'cargocar')", "precipitation", "(1, 0, 'cargocar')", "(1, 0, 'cargocar')", "hour", "(1, 0, 'cargocar')", "(1, 0, 'cargocar')", "precipitation", "(1, 0, 'cargocar')", "hour", "(1, 0, 'cargocar')", "(1, 0, 'cargocar')", "hour", "hour", "precipitation"]
#selected_col = ["hour", "precipitation", "dayofweek", "is_holiday"]

# Target predict column
predict_col = "(1, 0, 'tot')"

# Time shift backward for predict_col
dataset_shift_1 = dataset.shift(periods=1)[predict_col].fillna(method="bfill")
dataset_shift_2 = dataset.shift(periods=2)[predict_col].fillna(method="bfill")
dataset_shift_6 = dataset.shift(periods=6)[predict_col].fillna(method="bfill")

# Add time shifted predict_col to dataset
dataset = dataset.assign(predict_shift_1=pd.Series(dataset_shift_1).values)
dataset = dataset.assign(predict_shift_2=pd.Series(dataset_shift_2).values)
dataset = dataset.assign(predict_shift_6=pd.Series(dataset_shift_6).values)
#selected_col.append(predict_col)
#selected_col.append("predict_shift_1")
#selected_col.append("predict_shift_2")
#selected_col.append("predict_shift_6")

training_set = dataset[:-24*3*7]
predict_set = dataset[-24*3*7:][lambda df : ((df.hour >= 8) & (df.hour < 10)) | ((df.hour >= 17) & (df.hour < 19))]
#training_set[selected_col]
#predict_set[selected_col]

In [298]:
clf = SVR(C=1.0, epsilon=0.1)
clf.fit(training_set[selected_col], training_set[predict_col])
base_score = mape(clf.predict(predict_set[selected_col]), predict_set[predict_col])
print(base_score)

19.49573069464429


In [299]:
clf = SVR(C=1.0, epsilon=0.2)
for col in training_set.columns:
    selected_col.append(col)
    clf.fit(training_set[selected_col], training_set[predict_col])
    score = mape(clf.predict(predict_set[selected_col]), predict_set[predict_col])
    #score = clf.score(predict_set[selected_col], predict_set[predict_col])
    if score < base_score:
        print(col, score)
    selected_col.pop()

(1, 0, 'tot') 13.513572217888756
(1, 0, 'unknowncar') 13.513572217888756
